# Семинар 6 - Метрики, kNN


# Метрики

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
data = fetch_olivetti_faces()
data.target

In [ ]:
X, y = data.data, data.target

In [ ]:
indx_0 = np.where(y == 0)[0][:5]
indx_0

In [ ]:
indx_1 = np.where(y == 1)[0]
indx_1

In [ ]:
X = X[np.concatenate((indx_1, indx_0))]
y = y[np.concatenate((indx_1, indx_0))]

In [ ]:
print('В датасете {} объектов и {} признака'.format(X.shape[0], X.shape[1]))

Разделим выборку на две части: обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.5,
                                                    test_size=0.5, 
                                                    shuffle=True,
                                                   random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Зададим классификатор:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(x_train, y_train)
knn_predictons = knn.predict(x_test)

In [ ]:
preds = pd.DataFrame(y_test, columns=['True'])
preds['knn_pred'] = knn_predictons

In [ ]:
preds

## Accuracy

In [ ]:
# Посмотрим долю правильных ответов:
def accuracy(true, predictions):
    acc = sum(predictions==true)/len(predictions)
    return acc
accuracy(y_test, knn_predictons)

In [ ]:
#Тоже самое средставми sklearn:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_predictons)

## Precision

Посмотрим точность ответов (precision) без учета различных классов (всего TP/(всего TP + всего FP)):

In [ ]:
def precision(true, predictions):
    # Ваш код здесь:
    TP = 0
    FP = 0
    for i in range(len(true)):
        if true[i] == 1 and predictions[i] == 1:
            TP = TP + 1
        elif predictions [i] != true[i] and true[i] == 0:
            FP = FP + 1
    prec = TP/(TP+FP+np.finfo(float).eps)
    return prec
precision(y_test, knn_predictons)

Тоже самое средставми sklearn:

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, knn_predictons)

## Recall

Посмотрим полноту ответов (recall) 

In [ ]:
def recall(true, predictions):
    # Ваш код здесь:  
    TP = 0
    FP = 0
    for i in range(len(true)):
        if true[i] == 1 and predictions[i] == 1:
            TP = TP + 1
        elif predictions [i] == 0 and true[i] == 1:
            FP = FP + 1
    rec = TP/(TP+FP+np.finfo(float).eps)
    return rec
recall(y_test, knn_predictons)

Тоже самое средставми sklearn:

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, knn_predictons)

## F1_score

Посмотрим точность ответов (precision) без учета различных классов (всего TP/(всего TP + всего FP)):

In [ ]:
def F1_score(true, predictions):
    # Ваш код здесь: 
       
    f1 = 2*(recall(true, predictions)*precision(true, predictions))/(recall(true, predictions)+precision(true, predictions))
    return f1
F1_score(y_test, knn_predictons)

Тоже самое средставми sklearn:

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, knn_predictons)

## ROC curve:

Для этого, в отличае от других метрий, нам понадобятся не предсказания модели, а вероятности принадлежности классам, которые модель выдает.

In [ ]:
probs = knn.predict_proba(x_test)
probs

Также необходибо бинаризовать метки классов для подсчета TPR и FPR (только для мультикласса)

Считаем TPR и FPR

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, threshold = roc_curve(y_test, probs[:,1])
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

In [ ]:
def plot_roc_curve():
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for class 1')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_roc_curve()

Теперь постройте roc_curve для 0

# kNN - метод ближайших соседей

In [ ]:
from sklearn.datasets import load_digits

## Загрузим данные

In [ ]:
data = load_digits()
print(data['DESCR'])

In [ ]:
X, y = data.data, data.target

In [ ]:
print('В датасете {} объектов и {} признака'.format(X.shape[0], X.shape[1]))

### Посмотрим на объекты:

In [ ]:
i = np.random.randint(0, X.shape[0])
print('Class name: {}'.format(y[i]))
print(X[i].reshape(8,8))

X[i]
plt.imshow(X[i].reshape(8,8), cmap='gray_r')
plt.show()

Посмотрим на баланс классов:

In [ ]:
counts = np.unique(y, return_counts=True)
pd.DataFrame(counts[1], counts[0], columns=['counts']);
# sns.barplot(counts[0], counts[1])
# plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(counts[0], counts[1])
plt.show()

Разделим выборку на две части: обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.5,
                                                    test_size=0.5, 
                                                    shuffle=True,
                                                   random_state=18)
X_train.shape, X_test.shape, y_train.shape, y_test.shape  

## Метод ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Зададим классификатор:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train, y_train)
knn_predictons = knn.predict(X_test)

In [ ]:
preds = pd.DataFrame(y_test, columns=['True'])
preds['knn_pred'] = knn_predictons

In [ ]:
preds.head()

In [ ]:
# Посмотрим долю правильных ответов:
def accuracy(true, predictions):
    acc = predictions[true == predictions].shape[0]/true.shape[0]
    return acc
accuracy(y_test, knn_predictons)

In [ ]:
#Тоже самое средставми sklearn:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_predictons)

Постройте график зависимости доли верных ответов на тесте от количества соседей (от 1 до 100):

In [ ]:
# Ваш код здесь: 

## Поиск оптимальных параметров по сетке

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
n = np.linspace(1, 21, 21, dtype=int)

In [ ]:
kNN_cv = KNeighborsClassifier(n_neighbors=5)
params = { 
    'metric':['minkowski', 'manhattan'],
    'n_neighbors': n,
         }

gcv = GridSearchCV(kNN_cv, param_grid=params, cv=5, scoring='accuracy')
gcv.fit(X_train, y_train)

In [ ]:
def print_cv_results(a, len_gs, params, param_r, param_sep): 
    d = len(params['param_grid'][param_sep])
    ar=np.array(a).reshape(d,len_gs).T
    df=pd.DataFrame(ar)

    pen_par=params['param_grid'][param_sep]
    c_par=params['param_grid'][param_r]
    if type(c_par) != list: 
        c_par = c_par.tolist()
    columns_mapper=dict(zip(range(0, len(pen_par)),pen_par))
    row_mapper=dict(zip(range(0, len(c_par)), c_par))

    df.rename(columns=columns_mapper, index=row_mapper, inplace=True)

    plot = df.plot(title='Mean accuracy rating',grid=True)
    plot.set_xlabel(param_r, fontsize=13)
    plot.set_ylabel('acc', rotation=0, fontsize=13, labelpad=15)

In [ ]:
gcv.get_params()

In [ ]:
print_cv_results(gcv.cv_results_['mean_test_score'],
                 21, gcv.get_params(), 'n_neighbors','metric')
gcv.best_params_

In [ ]:
print('Лучший скор %.4f' % gcv.best_score_)
print('при метрике %(metric)s и %(n_neighbors)s соседей' % gcv.best_params_)

### Что получится на тесте?

In [ ]:
accuracy_score(y_test, gcv.predict(X_test))

In [ ]:
gcv_preds = pd.DataFrame(gcv.predict(X_test), columns=['kNN'])

In [ ]:
gcv_preds['True'] = y_test

In [ ]:
gcv_preds

Посмотрим на те цифры, которые "путает" наш классификатор

In [ ]:
gcv_preds[gcv_preds['True'] != gcv_preds['kNN']]

## Нормализация

Так как мы работаем с пикселями изображения - все признаки имеют "одинаковую" значимость.

P.S. Проводить нормализацию можно как для всего изображения (матрицы), так и для каждого пикселя отдельно.

Проведите нормировку признаков, вычислите точность, сделайте выводы

In [ ]:
# Ваш код здесь: 

Проведите стандартизацию признаков, вычислите точность, сделайте выводы

In [ ]:
# Ваш код здесь: 